In [1]:
%cd ..

/fs01/home/abbasgln/codes/medAI/projects/tta


In [2]:
import os
from dotenv import load_dotenv
# Loading environment variables
load_dotenv()

import torch
import torch.nn as nn
import typing as tp
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from dataclasses import dataclass, field
import logging
import wandb

import medAI
from medAI.utils.setup import BasicExperiment, BasicExperimentConfig

from utils.metrics import MetricCalculator

from timm.optim.optim_factory import create_optimizer

from einops import rearrange, repeat
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import timm

from copy import copy, deepcopy
import pandas as pd

from datasets.datasets import ExactNCT2013RFImagePatches
from medAI.datasets.nct2013 import (
    KFoldCohortSelectionOptions,
    LeaveOneCenterOutCohortSelectionOptions, 
    PatchOptions
)

from utils.sam_optimizer import SAM
from models.sar_model import SAR, configure_model, collect_params

In [3]:
LEAVE_OUT='PMCC'

## Data SAR

In [4]:
from sar_experiment import ExpConfig
config = ExpConfig(cohort_selection_config=LeaveOneCenterOutCohortSelectionOptions(leave_out=f"{LEAVE_OUT}"))

from baseline_experiment import BaselineConfig
from torchvision.transforms import v2 as T
from torchvision.tv_tensors import Image as TVImage


class Transform:
    def __init__(selfT, augment=False):
        selfT.augment = augment
        selfT.size = (256, 256)
    
    def __call__(selfT, item):
        patch = item.pop("patch")
        patch = copy(patch)
        patch = (patch - patch.min()) / (patch.max() - patch.min()) \
            if config.instance_norm else patch
        patch = TVImage(patch)
        # patch = T.ToImage()(patch)
        # patch = T.ToTensor()(patch)
        patch = T.Resize(selfT.size, antialias=True)(patch).float()
        
        
        if selfT.augment:
            # Augment support patches
            transform = T.Compose([
                T.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                T.RandomHorizontalFlip(p=0.5),
                T.RandomVerticalFlip(p=0.5),
            ])  
            patch = transform(patch)
        
        label = torch.tensor(item["grade"] != "Benign").long()
        return patch, label, item



# val_ds_sar = ExactNCT2013RFImagePatches(
#     split="val",
#     transform=Transform(),
#     cohort_selection_options=config.cohort_selection_config,
#     patch_options=config.patch_config,
#     debug=config.debug,
# )
        
test_ds_sar = ExactNCT2013RFImagePatches(
    split="test",
    transform=Transform(),
    cohort_selection_options=config.cohort_selection_config,
    patch_options=config.patch_config,
    debug=config.debug,
)


# val_loader_sar = DataLoader(
#     val_ds_sar, batch_size=config.batch_size, shuffle=False, num_workers=4
# )
test_loader_sar = DataLoader(
    test_ds_sar, batch_size=config.batch_size, shuffle=False, num_workers=4
)

Computing positions: 100%|██████████| 588/588 [00:10<00:00, 55.71it/s]


## Model

In [8]:
from baseline_experiment import FeatureExtractorConfig

fe_config = FeatureExtractorConfig()

# Create the model
model: nn.Module = timm.create_model(
    fe_config.model_name,
    num_classes=fe_config.num_classes,
    in_chans=1,
    features_only=fe_config.features_only,
    norm_layer=lambda channels: nn.GroupNorm(
                    num_groups=fe_config.num_groups,
                    num_channels=channels
                    ))

CHECkPOINT_PATH = os.path.join(os.getcwd(), f'logs/tta/baseline_gn_loco/baseline_gn_{LEAVE_OUT}_loco/checkpoints/', 'best_model.ckpt')

model.load_state_dict(torch.load(CHECkPOINT_PATH)['model'])
model.eval()
model.cuda()

a = True

## Run test SAR

In [12]:
loader = test_loader_sar

import math
metric_calculator = MetricCalculator()
desc = "test"

# Get adapt model
base_model = configure_model(deepcopy(model))
params, param_names = collect_params(base_model) # only affine params in norm layers
        
sam_optimizer = SAM(
    params,
    torch.optim.SGD,
    lr=config.optimizer_config.lr, 
    rho=0.05,
    momentum=0.9
    )
        
adapt_model = SAR(
    base_model,
    sam_optimizer,
    episodic=config.sar_config.episodic,
    steps=2,
    margin_e0=1*math.log(2),
    reset_constant_em=config.sar_config.reset_constant_em,
    )

criterion = nn.CrossEntropyLoss()

for i, batch in enumerate(tqdm(loader, desc=desc)):
    images, labels, meta_data = batch
    images = images.cuda()
    labels = labels.cuda()
    
    # Adapt to test

    logits = adapt_model(images)
    loss = criterion(logits, labels)
                    
    # Update metrics   
    metric_calculator.update(
        batch_meta_data = meta_data,
        probs = nn.functional.softmax(logits, dim=-1).detach().cpu(),
        labels = labels.detach().cpu(),
    )

test:   0%|          | 0/704 [00:00<?, ?it/s]

In [11]:
# Log metrics every epoch
metrics = metric_calculator.get_metrics()

# Update best score
(
    best_score_updated,
    best_score
    ) = metric_calculator.update_best_score(metrics, desc)

best_score_updated = copy(best_score_updated)
best_score = copy(best_score)
        
# Log metrics
metrics_dict = {
    f"{desc}/{key}": value for key, value in metrics.items()
    }
metrics_dict.update(best_score) if desc == "val" else None 


# wandb.log(
#     metrics_dict,
#     )
metrics_dict

{'test/patch_auroc': tensor(0.6670),
 'test/patch_accuracy': tensor(0.4827),
 'test/all_inv_patch_auroc': tensor(0.6372),
 'test/all_inv_patch_accuracy': tensor(0.4885),
 'test/core_auroc': tensor(0.7760),
 'test/core_accuracy': tensor(0.4434),
 'test/all_inv_core_auroc': tensor(0.7176),
 'test/all_inv_core_accuracy': tensor(0.4538)}

In [8]:
import wandb
name=f"sar_gn_{LEAVE_OUT}_loco_offline"
group=f"sar_gn_loco_offline"
wandb.init(project="tta", entity="mahdigilany", name=name, group=group)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: mahdigilany. Use `wandb login --relogin` to force relogin


In [9]:
metrics_dict.update({"epoch": 0})
wandb.log(
    metrics_dict,
    )
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁
test/all_inv_core_accuracy,▁
test/all_inv_core_auroc,▁
test/all_inv_patch_accuracy,▁
test/all_inv_patch_auroc,▁
test/core_accuracy,▁
test/core_auroc,▁
test/patch_accuracy,▁
test/patch_auroc,▁
epoch,0
test/all_inv_core_accuracy,0.46248
